In [97]:
import pandas as pd
import numpy as np

#### 1. Upload transactions.csv to Pandas and get a general data overview

In [170]:
data = pd.read_csv('transactions.csv')
data

,ArticleId,SiteId,Date,Quantity,Price,CostPrice
0,170936,4,9/05/2019,7,39.0,30.15
1,171979,7,18/01/2019,4,43.5,28.11
2,200838,5,16/05/2019,10,95.0,159.49
3,176139,7,21/12/2018,5,199.0,147.97
4,172551,5,11/04/2019,1,86.7,80.76
...,...,...,...,...,...,...
17992,170936,6,10/09/2018,5,39.0,30.06
17993,199219,6,23/01/2019,4,89.0,77.90
17994,165811,5,13/12/2018,4,21.9,15.74
17995,167108,8,1/04/2019,1,27.9,18.62


In [106]:
# Assign columns to Series
articleId =  data['ArticleId']
siteId    =  data['SiteId']
date      =  data['Date']
quantity  =  data['Quantity']
price     =  data['Price']
costPrice =  data['CostPrice']

In [168]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17997 entries, 0 to 17996
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   ArticleId  17997 non-null  int64  
 1   SiteId     17997 non-null  int64  
 2   Date       17997 non-null  object 
 3   Quantity   17997 non-null  int64  
 4   Price      17984 non-null  float64
 5   CostPrice  17970 non-null  float64
dtypes: float64(2), int64(3), object(1)
memory usage: 843.7+ KB


In [199]:
data.describe()
# Number of counts seems to differ between the columns: 17997 / 17997 / 17997 / 17984 / 17970
# Min. Quantity: -42 -> check abs

,ArticleId,SiteId,Quantity,Price,CostPrice
count,17997.000000,17997.00000,17997.000000,17984.000000,17970.000000
mean,177390.908763,5.99211,6.260599,81.778959,64.442784
std,12249.515240,1.45662,8.413260,72.251808,58.840182
min,165811.000000,4.00000,-42.000000,8.700000,5.830000
25%,170069.000000,5.00000,2.000000,26.900000,18.580000
50%,172551.000000,6.00000,4.000000,46.900000,30.130000
75%,176139.000000,7.00000,9.000000,95.000000,82.480000
max,200838.000000,9.00000,432.000000,399.000000,383.010000


In [52]:
# Double-check the number of records per each column
# Count of records between columns indeed differs.
data.count()

ArticleId    17997
SiteId       17997
Date         17997
Quantity     17997
Price        17984
CostPrice    17970
dtype: int64

#### 2. Check the count inconsistencies between columns

In [75]:
# Check 'Price' & 'CostPrice' for nulls.
# 13 rows are nulls for 'Price'. Therefore, count inconsistency of the column is caused by null-values (17997 - 13 = 17984).
# 27 rows are nulls for 'CostPrice'. Therefore count, inconsistency of the column is caused by null-values (17997 - 27 = 17970).
countPrice, countCostPrice = 0, 0
for i in data['Price'].isnull():
    if i == True:
        countPrice+=1
    else:
        continue

for i in data['CostPrice'].isnull():
    if i == True:
        countCostPrice+=1
    else:
        continue

countPrice, countCostPrice

(13, 27)

#### 3. Check the count by .groupby due to the declared shape

In [88]:
# '... Sample sales are based on real store data and are a cutout of the annual aggregation of 15 products (ArticleId) ...' 
# '... that were sold at 5 stores(SiteId) within two different groups ...'

groupByArticleId = data.groupby(data['ArticleId']).count()
groupBySiteId = data.groupby(data['SiteId']).count()

groupByArticleId, groupBySiteId
# ArticleId Check: 15/15 - correct
# SiteId Check: 6/15 - incorrect
# Issue №2. Either there is 1 extra SiteId or it should be 6 stores not 5. 
# (I guess 9th SiteId might be an extra one, since it holds only 253 records)

(           SiteId  Date  Quantity  Price  CostPrice
 ArticleId                                          
 165811       1560  1560      1560   1560       1560
 165954        302   302       302    301        301
 167108       1624  1624      1624   1623       1620
 169281        537   537       537    536        535
 170069        656   656       656    655        656
 170295       1185  1185      1185   1184       1185
 170936       1853  1853      1853   1853       1853
 171979       1012  1012      1012   1011       1008
 172551       1640  1640      1640   1639       1639
 172671        503   503       503    501        499
 174022       1801  1801      1801   1801       1800
 176139       1074  1074      1074   1072       1070
 194516        966   966       966    964        962
 199219       1604  1604      1604   1604       1602
 200838       1680  1680      1680   1680       1680,
         ArticleId  Date  Quantity  Price  CostPrice
 SiteId                                      

#### 4. Check dates

In [154]:
# Check date due to stated periods of time 
# (in head only 2018 & 2019 being shown, thus start from them, all the others go into else block)
x18, x19, xx = 0, 0, 0

for i in list(date):
    if i[-4:] == '2018':
        x18+=1
    elif i[-4:] == '2019':
        x19+=1
    else:
        xx+=1
        
x18,x19,xx,x18+x19+xx
# Total of 17997 with 8225 counts in 2018 & 9772 counts in 2019 + 0 counts for other years (data is for 18/19 only)
# Thus year range seems to be good (2-3 years range with no 'odd' dates such as 1900, etc.)
# + all the dates end with correct years, there are no dates, which e.g. end with months or days

(8225, 9772, 0, 17997)

In [159]:
lenLess9, len9or10, lenMore10, lenOther = 0, 0, 0, 0
for i in date:
    if len(i) < 9:
        lenLess9+=1
    elif len(i) == 9 or len(i) == 10:
        len9or10+=1
    elif len(i) > 10:
        lenMore10+=1
    else:
        lenOther+=1
lenLess9,len9or10,lenMore10,lenOther,lenLess9+len9or10+lenMore10+lenOther
# Date lenghts seems to be OK for all records (9 or 10 len -> 9 when zeros in mono-numbers omitted)

(0, 17997, 0, 0, 17997)

In [172]:
dd, mm = 0, 0
for i in date:
    if int(i.split('/')[0]) > 31:
        dd+=1
    if int(i.split('/')[1]) > 12:
        mm+=1

dd, mm
# days / month seems to be correct (no records w/ months > 12, days > 31)
# also max days per each month can be checked to assure the correct assignment: 
# e.g. February can have max 29 days thus DD cannot be > 29 as for March, which can have 31 days

(0, 0)

In [94]:
data.dtypes

ArticleId      int64
SiteId         int64
Date          object
Quantity       int64
Price        float64
CostPrice    float64
dtype: object

#### 5. Check abs/0 of Qty/Price/CostPrice

In [195]:
minus, zero, plus = 0, 0, 0
for i in quantity:
    if int(i) == 0:
        zero+=1
    elif int(i) < 0:
        minus+=1
    elif int(i) > 0:
        plus+=1

minus, zero, plus  
# 820,  0,  17177  qty has to be absolute

(820, 0, 17177)

In [197]:
minus, zero, plus = 0, 0, 0
for i in price:
    if float(i) == 0:
        zero+=1
    elif float(i) < 0:
        minus+=1
    elif float(i) > 0:
        plus+=1

minus, zero, plus  
# 0,    0,  17984  except blanks, price is ok (all abs)

(0, 0, 17984)

In [198]:
minus, zero, plus = 0, 0, 0
for i in costPrice:
    if float(i) == 0:
        zero+=1
    elif float(i) < 0:
        minus+=1
    elif float(i) > 0:
        plus+=1
        
minus, zero, plus
# 0,    0,  17970  except blanks, costPrice is ok (all abs)

(0, 0, 17970)

#### 6. Return clean CSV

In [190]:
# articleId =  data['ArticleId']
# siteId    =  data['SiteId']
# date      =  data['Date']
# quantity  =  data['Quantity']
# price     =  data['Price']
# costPrice =  data['CostPrice']

out = {'ArticleId': articleId.astype(int), 
'SiteId': siteId.astype(int),
'Date': pd.to_datetime(date),
'Quantity': quantity.astype(int), 
'Price': pd.to_numeric(price),
'CostPrice': pd.to_numeric(costPrice)}

outDf = pd.DataFrame(data=out)
outDf.to_csv('out.csv')

# Here only datatypes fixed as an example...